# Function: Monte Carlo Stock Price Generation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Monte Carlo Stock Price Simulation for configurable intervals
def generate_stock_price_data(interval=5, hours=23):
    """
    Generate stock price data for specified hours using Monte Carlo method
    
    Parameters:
    -----------
    interval : int, default=5
        Time interval in seconds between each price point
    hours : int, default=23
        Number of hours to simulate (6 PM to 5 PM next day = 23 hours)
    """
    # Parameters
    initial_price = 100.0  # Starting stock price
    drift = 0.0001  # Daily drift (scaled for interval)
    volatility = 0.02  # Daily volatility
    
    # Calculate time step and number of steps based on interval
    time_step = interval / (24 * 60 * 60)  # Convert seconds to fraction of day
    num_steps = int(hours * 60 * 60 / interval)  # Total intervals in specified hours
    
    # Generate time series starting at 6 PM (18:00)
    start_time = datetime.now().replace(hour=18, minute=0, second=0, microsecond=0)
    timestamps = [start_time + timedelta(seconds=i*interval) for i in range(num_steps)]
    
    # Monte Carlo simulation using Geometric Brownian Motion
    prices = [initial_price]
    
    for i in range(1, num_steps):
        # Random shock (normal distribution)
        random_shock = np.random.normal(0, 1)
        
        # Calculate price change using GBM formula
        price_change = drift * time_step + volatility * np.sqrt(time_step) * random_shock
        
        # Calculate new price
        new_price = prices[-1] * np.exp(price_change)
        prices.append(new_price)
    
    # Create DataFrame
    stock_data = pd.DataFrame({
        'timestamp': timestamps,
        'price': prices
    })
    
    return stock_data


# Function: Database Stock Price Generation

In [ ]:
import pandas as pd
import sqlite3

# Connect to aia_big.db and tell what data is there
def connect_to_database(db_path='aia_big.db'):
    """
    Connect to the SQLite database and return the connection object.
    
    Parameters:
    -----------
    db_path : str, default='aia_big.db'
        Path to the SQLite database file.
    """

    conn = sqlite3.connect(db_path)
    return conn

def get_table_info(conn):
    """
    Get information about the tables in the database.
    
    Parameters:
    -----------
    conn : sqlite3.Connection
        Connection object to the SQLite database.
    
    Returns:
    --------
    list of tuples containing table names and their row counts.
    """
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    table_info = []
    for table in tables:
        cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
        row_count = cursor.fetchone()[0]
        table_info.append((table[0], row_count))
    
    return table_info

def fetch_table_data(conn, table_name):
    """
    Fetch data from a specific table in the database.
    
    Parameters:
    -----------
    conn : sqlite3.Connection
        Connection object to the SQLite database.
    table_name : str
        Name of the table to fetch data from.
    
    Returns:
    --------
    pandas DataFrame containing the table data.
    """
    query = f"SELECT * FROM {table_name}"
    return pd.read_sql_query(query, conn)

def close_database_connection(conn):
    """
    Close the database connection.
    
    Parameters:
    -----------
    conn : sqlite3.Connection
        Connection object to the SQLite database.
    """
    conn.close()

# Connect to the database
conn = connect_to_database()

# Get table information
table_info = get_table_info(conn)
print("Tables in the database:", table_info)

# Get column names and types for each table
for table in table_info:
    table_name = table[0]
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    print(f"Columns in {table_name}: {[col[1] for col in columns]}")

# Execute the following SQL query to get the first 5 rows of each table
for table in table_info:
    table_name = table[0]
    query = f"SELECT * FROM {table_name} LIMIT 5"
    df = pd.read_sql_query(query, conn)
    print(f"First 5 rows of {table_name}:\n", df)

# # Fetch data from a specific table (e.g., 'stock_prices')
# if table_info:
#     table_name = table_info[0][0]  # Get the first table name
#     data = fetch_table_data(conn, table_name)
#     print(f"Data from {table_name}:", data.head())

# Close the database connection
close_database_connection(conn)

# Function: OANDA Stock Price Generation

In [126]:
# OANDA Live FXTrade API Connection
import requests
import pandas as pd
from datetime import datetime, timedelta
import json

def get_oanda_data(api_key, account_id, instrument='USD_CAD', granularity='S5', hours=5):
    """
    Connect to OANDA live fxtrade environment and fetch real market data
    
    Parameters:
    -----------
    api_key : str
        Your OANDA API key from live account
    account_id : str
        Your OANDA live account ID
    instrument : str, default='USD_CAD'
        Currency pair to fetch
    granularity : str, default='S5'
        Time granularity (S5 = 5 seconds)
    hours : int, default=10
        Number of hours of historical data to fetch
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with real market data
    """
    
    # LIVE OANDA API URL (NOT practice!)
    BASE_URL = "https://api-fxtrade.oanda.com"
    
    print("🔴 CONNECTING TO OANDA LIVE FXTRADE ENVIRONMENT")
    print("=" * 55)
    print("⚠️  WARNING: This will connect to LIVE market data")
    print(f"📊 Requesting: {instrument} | {granularity} | Last {hours} hours")
    print("=" * 55)
    
    # Validate inputs
    if not api_key or api_key == "your_live_api_key_here":
        print("❌ ERROR: Live API key is required!")
        print("\n🔧 TO GET YOUR LIVE OANDA CREDENTIALS:")
        print("1. Log into your OANDA account at: https://www.oanda.com/")
        print("2. Go to 'Manage API Access' in account settings")
        print("3. Generate a Personal Access Token")
        print("4. Copy your Account ID from account overview")
        print("\n💡 USAGE:")
        print("live_data = connect_oanda_live(")
        print("    api_key='your_actual_api_key',")
        print("    account_id='your_actual_account_id'")
        print(")")
        return None
    
    if not account_id or account_id == "your_live_account_id_here":
        print("❌ ERROR: Live Account ID is required!")
        return None
    
    # Headers for API request
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    
    # Calculate count based on granularity and hours
    if granularity == 'S5':
        count = min(hours * 60 * 12, 5000)  # 12 five-second intervals per minute, max 5000
    elif granularity == 'S10':
        count = min(hours * 60 * 6, 5000)   # 6 ten-second intervals per minute
    elif granularity == 'M1':
        count = min(hours * 60, 5000)       # 60 one-minute intervals per hour
    elif granularity == 'M5':
        count = min(hours * 12, 5000)       # 12 five-minute intervals per hour
    else:
        count = min(7200, 5000)  # Default fallback
    
    # API endpoint for historical candles
    url = f"{BASE_URL}/v3/instruments/{instrument}/candles"
    
    # Parameters for the request
    params = {
        'count': count,
        'granularity': granularity#,
        # 'price': 'MBA',  # Mid, Bid, Ask prices
        # 'includeFirst': 'true'
    }
    
    try:
        print(f"🌐 Making API request to OANDA live servers...")
        print(f"   URL: {url}")
        print(f"   Params: {params}")
        
        # Make the API request
        response = requests.get(url, headers=headers, params=params, timeout=30)
        
        # Check for HTTP errors
        if response.status_code == 401:
            print("❌ AUTHENTICATION ERROR (401)")
            print("   • Check your API key is correct")
            print("   • Ensure your API key has proper permissions")
            print("   • Verify you're using the live account API key")
            return None
        elif response.status_code == 403:
            print("❌ FORBIDDEN ERROR (403)")
            print("   • Your account may not have API access enabled")
            print("   • Check if your account is verified and funded")
            return None
        elif response.status_code == 404:
            print("❌ NOT FOUND ERROR (404)")
            print(f"   • Check instrument name: {instrument}")
            print(f"   • Check granularity: {granularity}")
            return None
        elif response.status_code != 200:
            print(f"❌ HTTP ERROR {response.status_code}")
            print(f"   Response: {response.text}")
            return None
        
        # Parse JSON response
        data = response.json()
        
        if 'candles' not in data:
            print("❌ ERROR: No candles data in response")
            print(f"Response: {data}")
            return None
        
        candles = data['candles']
        print(f"✅ Successfully received {len(candles)} candles from OANDA live")
        
        # Convert to DataFrame
        market_data = []
        for candle in candles:
            # Convert timestamp to New York timezone and remove timezone info
            timestamp = pd.to_datetime(candle['time'])
            # Convert to New York timezone
            timestamp = timestamp.tz_convert('America/New_York')
            # Remove timezone info (localize to None)
            timestamp = timestamp.tz_localize(None)
            
            # Extract OHLC data
            mid = candle.get('mid', {})
            bid = candle.get('bid', {})
            ask = candle.get('ask', {})
            
            if not mid:
                continue  # Skip if no mid prices
            
            # Get prices
            open_price = float(mid['o'])
            high_price = float(mid['h'])
            low_price = float(mid['l'])
            close_price = float(mid['c'])
            
            bid_price = float(bid.get('c', close_price - 0.0001))
            ask_price = float(ask.get('c', close_price + 0.0001))
            
            # Calculate spread in pips (for USD/CAD, 1 pip = 0.0001)
            spread_pips = (ask_price - bid_price) * 10000
            
            market_data.append({
                'timestamp': timestamp,
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price,
                'mid': close_price,
                'bid': bid_price,
                'ask': ask_price,
                'volume': candle.get('volume', 0),
                'spread_pips': round(spread_pips, 1),
                'complete': candle.get('complete', True)
            })
        
        if not market_data:
            print("❌ ERROR: No valid market data received")
            return None
        
        # Create DataFrame
        df = pd.DataFrame(market_data)
        
        # Add price column for compatibility with EMA functions
        df['price'] = df['close']
        
        # Sort by timestamp to ensure chronological order
        df = df.sort_values('timestamp').reset_index(drop=True)
        
        print(f"\n📊 LIVE MARKET DATA SUMMARY:")
        print(f"   • Instrument: {instrument}")
        print(f"   • Granularity: {granularity}")
        print(f"   • Total candles: {len(df):,}")
        print(f"   • Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")
        print(f"   • Price range: {df['close'].min():.5f} - {df['close'].max():.5f}")
        print(f"   • Current price: {df['close'].iloc[-1]:.5f}")
        print(f"   • Average spread: {df['spread_pips'].mean():.1f} pips")
        
        # Show latest data
        print(f"\n📈 LATEST 3 CANDLES:")
        latest_cols = ['timestamp', 'open', 'high', 'low', 'close', 'bid', 'ask', 'spread_pips']
        print(df[latest_cols].tail(3).to_string(index=False, float_format='%.5f'))
        

        # return the dataframe with timestamp and  price columns
        return df[['timestamp', 'price']]
        
    except requests.exceptions.Timeout:
        print("❌ TIMEOUT ERROR: Request took too long")
        print("   • Try again or reduce the number of hours")
        return None
    except requests.exceptions.ConnectionError:
        print("❌ CONNECTION ERROR: Cannot reach OANDA servers")
        print("   • Check your internet connection")
        print("   • OANDA servers may be temporarily unavailable")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ REQUEST ERROR: {e}")
        return None
    except json.JSONDecodeError:
        print("❌ JSON ERROR: Invalid response from OANDA")
        print(f"   Response: {response.text[:200]}...")
        return None
    except Exception as e:
        print(f"❌ UNEXPECTED ERROR: {e}")
        return None

# <font color=red>Function:</font> IB Stock Price Generation

In [ ]:
# Connect to Interactive Brokers (IB) for historical data at 5 second intervals
def connect_ib_historical_data(symbol='USD.CAD', duration='10H', bar_size='5 secs'):
    """
    Connect to Interactive Brokers (IB) for historical data at 5 second intervals.
    
    Parameters:
    -----------
    symbol : str, default='USD.CAD'
        The symbol to fetch data for
    duration : str, default='10H'
        Duration of historical data to fetch (e.g., '10H' for 10 hours)
    bar_size : str, default='5 secs'
        Size of each bar (e.g., '5 secs')
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with historical data
    """
    # Placeholder for IB connection logic
    print(f"Connecting to IB for {symbol} with duration {duration} and bar size {bar_size}")
    
    # Simulate fetching data (replace with actual IB API call)
    timestamps = pd.date_range(start=datetime.now() - timedelta(hours=10), periods=1200, freq='5S')
    prices = np.random.normal(loc=1.25, scale=0.01, size=len(timestamps))  # Simulated prices
    
    df = pd.DataFrame({'timestamp': timestamps, 'price': prices})
    
    return df

# <font color=red>Function:</font> CSV Stock Price Generation

In [ ]:
# Read CSV stock price data to DataFrame
def read_csv_stock_data(file_path):
    """
    Read stock price data from a CSV file into a DataFrame.
    
    Parameters:
    -----------
    file_path : str
        Path to the CSV file containing stock price data.
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with stock price data.
    """
    try:
        df = pd.read_csv(file_path)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)
        return df
    except Exception as e:
        print(f"❌ ERROR reading CSV file: {e}")
        return None     


# Function: Transform 5 second data to variable interval data

In [ ]:
def create_variable_interval_data(base_df, min_interval, max_interval):
    """
    Create a new DataFrame with variable time intervals by randomly removing rows
    
    Parameters:
    -----------
    base_df : pandas.DataFrame
        Base DataFrame with 'timestamp' and 'price' columns
    min_interval : int, default=1
        Minimum time interval in seconds
    max_interval : int, default=60
        Maximum time interval in seconds
    
    Returns:
    --------
    pandas.DataFrame
        New DataFrame with variable time intervals
    """
    
    # Start with first row
    variable_data = [base_df.iloc[0]]
    current_index = 0
    
    while current_index < len(base_df) - 1:
        # Generate random interval between min and max seconds
        random_interval = np.random.randint(min_interval, max_interval + 1)
        
        # Move forward by this interval (in seconds)
        next_index = current_index + random_interval
        
        # Make sure we don't exceed the DataFrame length
        if next_index < len(base_df):
            variable_data.append(base_df.iloc[next_index])
            current_index = next_index
        else:
            # Add the last row if we haven't reached it
            if current_index != len(base_df) - 1:
                variable_data.append(base_df.iloc[-1])
            break
    
    # Create new DataFrame
    variable_df = pd.DataFrame(variable_data).reset_index(drop=True)
    
    return variable_df


# Function: Calculate Data Frame's time based EMAs (including TEMA)

In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta

def calculate_time_based_ema(stock_df, type=1, period_minutes=15):
    """
    Calculate Exponential Moving Average based on actual time period (minutes) using timestamps.
    
    Parameters:
    -----------
    stock_df : pandas.DataFrame
        DataFrame containing 'timestamp' and 'price' columns
    period_minutes : int, default=15
        The time period in minutes for the EMA calculation
    
    Returns:
    --------
    pandas.Series
        Series containing the EMA values aligned with the input DataFrame index
    
    Notes:
    ------
    - This function is truly time-based and uses actual timestamps, not row counts
    - The smoothing factor (alpha) is calculated based on the actual time period
    - Works with any data frequency (1-second, 5-second, 1-minute, etc.)
    - EMA formula: EMA_today = (Price_today * α) + (EMA_yesterday * (1 - α))
    - Where α = 2 / (N + 1), and N is calculated from the actual time period
    """
    
    # Validate input DataFrame
    if not isinstance(stock_df, pd.DataFrame):
        raise ValueError("stock_df must be a pandas DataFrame")
    
    if 'timestamp' not in stock_df.columns or 'price' not in stock_df.columns:
        raise ValueError("DataFrame must contain 'timestamp' and 'price' columns")
    
    if len(stock_df) == 0:
        return pd.Series(dtype=float)
    
    if len(stock_df) < 2:
        # If only one data point, return the price itself
        return pd.Series([stock_df['price'].iloc[0]], index=stock_df.index)
    
    # Calculate the actual time interval between data points (in seconds)
    time_diffs = stock_df['timestamp'].diff().dropna()
    avg_interval_seconds = time_diffs.dt.total_seconds().mean()
    
    # Calculate the equivalent number of periods for the specified time window
    period_seconds = period_minutes * 60
    equivalent_periods = period_seconds / avg_interval_seconds
    
    # Calculate smoothing factor (alpha) based on equivalent periods
    # Using standard EMA formula: α = 2 / (N + 1)
    alpha = 2.0 / (equivalent_periods + 1)
    
    # Initialize EMA series
    ema_values = []
    dema_values = []
    tema_values = []
    purple_values = []
    
    # First EMA value is the first price
    first_price = stock_df['price'].iloc[0]
    ema_values.append(first_price)
    dema_values.append(first_price)
    tema_values.append(first_price)
    purple_values.append(first_price)  # Initial purple value
    
    # Calculate EMA for subsequent values using time-based smoothing
    for i in range(1, len(stock_df)):
        current_price = stock_df['price'].iloc[i]
        previous_ema = ema_values[-1]
        previous_dema = dema_values[-1]
        previous_tema = tema_values[-1]

        # Calculate time elapsed since previous data point
        time_elapsed = (stock_df['timestamp'].iloc[i] - stock_df['timestamp'].iloc[i-1]).total_seconds()
        
        # Adjust alpha based on actual time elapsed vs expected interval
        # This ensures consistency regardless of irregular timestamps
        time_weight = time_elapsed / avg_interval_seconds
        adjusted_alpha = 1 - (1 - alpha) ** time_weight
        
        # EMA formula with time-adjusted smoothing
        current_ema = adjusted_alpha * current_price + (1 - adjusted_alpha) * previous_ema
        current_dema = adjusted_alpha * current_ema + (1 - adjusted_alpha) * previous_dema
        current_tema = adjusted_alpha * current_dema + (1 - adjusted_alpha) * previous_tema
        
        ema_values.append(current_ema)
        dema_values.append(current_dema)
        tema_values.append(current_tema)
        purple_values.append(3 * (current_ema - current_dema) + current_tema)
    
    # Return as pandas Series with the same index as input DataFrame
    if type == 1:
        return pd.Series(ema_values, index=stock_df.index, name=f'EMA_{period_minutes}min')
    elif type == 2:
        return pd.Series(dema_values, index=stock_df.index, name=f'DEMA_{period_minutes}min')
    elif type == 3:
        return pd.Series(purple_values, index=stock_df.index, name=f'TEMA_{period_minutes}min')
    else:
        raise ValueError("Invalid type specified. Use 1 for EMA, 2 for DEMA, or 3 for TEMA.")


# <font color=lightgreen>Function:</font> OANDA Get Instrument Precision

In [ ]:
import json
import requests

def get_instrument_precision(url, api_key, account_id, instrument_name):
    """
    Retrieves the display precision (decimal places) for a financial instrument from OANDA.

    Makes an authenticated request to the OANDA API to get instrument details and 
    returns the number of decimal places used for price display.

    Args:
        url (str): The OANDA API base URL (e.g., 'https://api-fxtrade.oanda.com').
        api_key (str): Your OANDA API key.
        account_id (str): Your OANDA account ID.
        instrument_name (str): The instrument name (e.g., 'EUR_USD', 'USD_CAD').

    Returns:
        int: Number of decimal places for price display, or None if instrument not found or error occurs.
    """

    endpoint = f"{url}/v3/accounts/{account_id}/instruments"
    headers = {'Authorization': f'Bearer {api_key}'}
    params = None
    data = None
    
    try:
        response = requests.get(endpoint, headers=headers, params=params, data=data)
        response.raise_for_status()  # Raise an exception for bad status codes
        
        instruments = response.json()['instruments']
        
        for instrument in instruments:
            if instrument['name'] == instrument_name:
                return instrument['displayPrecision']
                
        return None # Instrument not found
        
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# <font color=lightgreen>Function:</font> OANDA Create Stop Order

In [ ]:
import json
import requests

def create_stop_order(url, api_key, account_id, instrument_name, units, price):
    """
    Creates a stop order for a financial instrument using the OANDA REST API.

    A stop order is triggered when the market price reaches the specified price level.
    For buy orders, the stop price should be above the current market price.
    For sell orders, the stop price should be below the current market price.
    Args:
        url (str): The OANDA API base URL (e.g., 'https://api-fxtrade.oanda.com').
        api_key (str): Your OANDA API key.
        account_id (str): Your OANDA account ID.
        instrument_name (str): The instrument to trade (e.g., 'EUR_USD', 'USD_CAD').
        units (int): Number of units to trade. Positive for buy orders, negative for sell orders.
        price (float): The stop price at which the order will be triggered.

    Returns:
        dict or None: JSON response from OANDA API containing order details, or None if error occurs.
    """

    endpoint = f"{url}/v3/accounts/{account_id}/orders"
    headers = {'Authorization': f'Bearer {api_key}'}
    params = None
    data = None
    json = {
        "order": {
            "price": str(price),
            "instrument": instrument_name,
            "units": str(units),
            "type": "STOP",
            "timeInForce": "GTC"
        }
    }

    try:
        response = requests.post(endpoint, headers=headers, params=params, data=data, json=json)
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None



# Function: List orders

In [ ]:
import json
import requests

def list_orders(url, api_key, account_id):
    endpoint = f'{url}/v3/accounts/{account_id}/orders'
    headers = {'Authorization': f'Bearer {api_key}'}
    params = None
    data = None
    json = None
    try:
        response = requests.get(endpoint, headers=headers, params=params, data=data, json=json)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

    orders = response.json().get('orders', [])
    for order in orders:
        if 'instrument' in order:
            print(order['instrument'], order['id'], order['price'], order['type'])
    
    # return list of order ids
    return [order['id'] for order in orders if 'id' in order]


# Function: List trades

In [ ]:
import json
import requests

def list_trades(url, api_key, account_id):
    endpoint = f'{url}/v3/accounts/{account_id}/trades'
    headers = {'Authorization': f'Bearer {api_key}'}
    params = None
    data = None
    json = None
    try:
        response = requests.get(endpoint, headers=headers, params=params, data=data, json=json)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

    trades = response.json().get('trades', [])
    for trade in trades:
        if 'instrument' in trade:
            print(trade['instrument'], trade['id'], trade['price'], trade['currentUnits'], trade['state'])
    return [trade['id'] for trade in trades if 'id' in trade]

# <font color=lightgreen>Function:</font> OANDA Cancel Order(s) for entire account <font color=red>TODO:</font> For a given instrument

In [ ]:
import json
import requests
import numpy as np

def cancel_order(url, api_key, account_id, order_id):
    """
    Cancels a specific order using the OANDA REST API.

    Makes a PUT request to the OANDA API to cancel an existing order by its ID.
    Once cancelled, the order will no longer be active and cannot be executed.

    Args:
        url (str): The OANDA API base URL (e.g., 'https://api-fxtrade.oanda.com').
        api_key (str): Your OANDA API key.
        account_id (str): Your OANDA account ID.
        order_id (str): The unique ID of the order to cancel.

    Returns:
        None: Function doesn't return the response data (commented out), 
              or None if an error occurs.
    """
    endpoint = f"{url}/v3/accounts/{account_id}/orders/{order_id}/cancel"
    headers = {'Authorization': f'Bearer {api_key}'}
    params = None
    data = None
    json = None
    try:
        response = requests.put(endpoint, headers=headers, params=params, data=data, json=json)
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def cancel_orders(url, api_key, account_id, side=0):
    """
    Cancels multiple orders for a specific instrument using the OANDA REST API.

    Retrieves all orders for the account, filters them by instrument and order type,
    then cancels orders based on the specified side parameter. Only cancels
    MARKET_IF_TOUCHED and STOP orders, excluding STOP_LOSS and TRAILING_STOP_LOSS.

    Args:
        url (str): The OANDA API base URL (e.g., 'https://api-fxtrade.oanda.com').
        api_key (str): Your OANDA API key.
        account_id (str): Your OANDA account ID.
        instrument_name (str): The instrument to cancel orders for (e.g., 'EUR_USD', 'USD_CAD').
        side (int, optional): Direction filter for orders to cancel:
                             -1 to cancel sell orders (negative units),
                             1 to cancel buy orders (positive units),
                             0 to cancel all qualifying orders.
                             Default is 0.

    Returns:
        None: Function doesn't return data but prints order information and 
              calls cancel_order() for each qualifying order.

    Note:
        This function will print all orders and their types for debugging purposes.
        It specifically excludes STOP_LOSS and TRAILING_STOP_LOSS orders from cancellation.
    """
    endpoint = f'{url}/v3/accounts/{account_id}/orders'
    headers = {'Authorization': f'Bearer {api_key}'}
    params = None
    data = None
    json = None
    try:
        response = requests.get(endpoint, headers=headers, params=params, data=data, json=json)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

    orders = response.json().get('orders', [])

    if orders:
        for order in orders:
            if order['type'] not in ['STOP_LOSS','TRAILING_STOP_LOSS']:
                if order['type'] in ['MARKET_IF_TOUCHED', 'STOP']:
                    if side >= 0:
                        if np.sign(float(order['units'])) > 0:
                            cancel_order(url, api_key, account_id, order['id'])
                            print(f"Cancelled order: {order['id']} | Type: {order['type']} | Units: {order['units']} | Price: {order['price']}")
                    if side <= 0:
                        if np.sign(float(order['units'])) < 0:
                            cancel_order(url, api_key, account_id, order['id'])
                            print(f"Cancelled order: {order['id']} | Type: {order['type']} | Units: {order['units']} | Price: {order['price']}")
                    

# <font color=lightgreen>Function:</font> OANDA Get Instrument Position

In [ ]:
def get_instrument_position(url, api_key, account_id, instrument_name):
    """
    Retrieves the number of units for a given instrument.

    Args:
        api_key (str): Your Oanda API key.
        account_id (str): Your Oanda account ID.
        instrument (str): The instrument to check (e.g., 'EUR_USD').

    Returns:
        int: The number of units of the instrument. Positive for long, negative for short, 0 for no position.
    """

    params = {'instruments': instrument_name}
    headers = {'Authorization': f'Bearer {api_key}'}
    endpoint = f'{url}/v3/accounts/{account_id}/positions/{instrument_name}'
    data = None
    json = None
    try:
        response = requests.get(endpoint, headers=headers, params=params, data=data, json=json)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None
    json = response.json()
    if 'position' not in json:
        print(f"❌ ERROR: No position data found for instrument {instrument_name}")
        return None
    position = json['position']
    if 'long' in position and 'short' in position:
        long_units = int(position['long']['units'])
        short_units = int(position['short']['units'])
        total_units = long_units + short_units
        print(f"Position for {instrument_name}: Long={long_units}, Short={short_units}, Total={total_units}")
        return total_units
    else:
        print(f"❌ ERROR: Position data for {instrument_name} is incomplete or missing long/short information")
        return None




# Function: OANDA Live Streaming Prices

In [ ]:
import requests
import json
import time
from datetime import datetime
import threading
from queue import Queue
import json

# Load secrets from secrets.json
with open('secrets.json', 'r') as f:
    secrets = json.load(f)

api_key = secrets['api_key']
account_id = secrets['account_id']

def stream_oanda_live_prices(api_key, account_id, instrument='USD_CAD', callback=None, max_duration=None):
    """
    Stream live prices from OANDA API for a single instrument
    
    Parameters:
    -----------
    api_key : str
        Your OANDA live API key
    account_id : str  
        Your OANDA live account ID
    instrument : str, default='USD_CAD'
        Single instrument to stream (e.g., 'EUR_USD', 'GBP_JPY')
    callback : function, optional
        Function to call with each price update: callback(timestamp, instrument, bid, ask, price)
    max_duration : int, optional
        Maximum streaming duration in seconds (None = unlimited)
    
    Returns:
    --------
    generator or None
        Yields price dictionaries or None if connection fails
    """
    
    # LIVE OANDA STREAMING API URL
    STREAM_URL = "https://stream-fxtrade.oanda.com"
    
    print("🔴 CONNECTING TO OANDA LIVE STREAMING")
    print("=" * 50)
    print("⚠️  WARNING: This connects to LIVE market stream")
    print(f"📊 Streaming: {instrument}")
    print(f"⏱️  Duration: {'Unlimited' if max_duration is None else f'{max_duration}s'}")
    print("=" * 50)
    
    # Validate inputs
    if not api_key:
        print("❌ ERROR: Live API key is required!")
        return None
    
    if not account_id:
        print("❌ ERROR: Live Account ID is required!")
        return None
    
    # Headers for streaming request
    headers = {
        'Authorization': f'Bearer {api_key}',
        'Accept': 'application/stream+json',
        'Content-Type': 'application/json'
    }
    
    # Streaming endpoint for prices
    url = f"{STREAM_URL}/v3/accounts/{account_id}/pricing/stream"
    
    # Parameters for streaming
    params = {
        'instruments': instrument,
        'snapshot': 'true'  # Include initial snapshot
    }
    
    try:
        print(f"🌐 Initiating streaming connection...")
        print(f"   URL: {url}")
        print(f"   Instrument: {instrument}")
        
        # Get instrument precision
        precision = get_instrument_precision(instrument)

        # Make streaming request
        response = requests.get(url, headers=headers, params=params, stream=True, timeout=30)
        
        # Check for HTTP errors
        if response.status_code == 401:
            print("❌ AUTHENTICATION ERROR (401)")
            print("   • Check your API key is correct")
            print("   • Ensure your API key has streaming permissions")
            return None
        elif response.status_code == 403:
            print("❌ FORBIDDEN ERROR (403)")
            print("   • Your account may not have streaming access")
            print("   • Check if your account is verified and funded")
            return None
        elif response.status_code == 404:
            print(f"❌ NOT FOUND ERROR (404)")
            print(f"   • Check instrument name: {instrument}")
            return None
        elif response.status_code != 200:
            print(f"❌ HTTP ERROR {response.status_code}")
            print(f"   Response: {response.text}")
            return None
        
        print("✅ Streaming connection established!")
        print("📈 Receiving live price updates...")
        print("   Press Ctrl+C to stop streaming")
        print("-" * 50)
        
        start_time = time.time()
        price_count = 0
        previous_price = None
        
        # Process streaming data line by line
        for line in response.iter_lines():
            # Check duration limit
            if max_duration and (time.time() - start_time) > max_duration:
                print(f"\n⏰ Reached maximum duration of {max_duration} seconds")
                break
                
            if line:
                try:
                    # Parse JSON data
                    data = json.loads(line.decode('utf-8'))
                    
                    # Handle different types of messages
                    if data.get('type') == 'PRICE':
                        timestamp = datetime.now()
                        
                        # Extract price information
                        instrument_name = data.get('instrument', instrument)
                        
                        # Get bid/ask prices
                        bids = data.get('bids', [])
                        asks = data.get('asks', [])
                        
                        if bids and asks:
                            bid_price = round(float(bids[0]['price']), precision)
                            ask_price = round(float(asks[0]['price']), precision)   
                            mid_price = round((bid_price + ask_price) / 2, precision)
                            spread_pips = (ask_price - bid_price) * 10000  # For most pairs
                            
                            # Skip if price hasn't changed
                            if previous_price is not None and bid_price == previous_price:
                                continue
                            
                            # Update previous price
                            previous_price = bid_price
                            price_count += 1
                            
                            # Create price dictionary
                            price_data = {
                                'timestamp': timestamp,
                                'instrument': instrument_name,
                                'bid': bid_price,
                                'ask': ask_price,
                                'price': mid_price,  # Mid price for compatibility
                                'spread_pips': round(spread_pips, 1),
                                'time': data.get('time', timestamp.isoformat()),
                                'tradeable': data.get('tradeable', True)
                            }
                            
                            # Print price update (every 10th update to avoid spam)
                            if price_count % 10 == 0:
                                print(f"💰 {timestamp.strftime('%H:%M:%S')} | {instrument_name} | "
                                      f"Bid: {bid_price:.5f} | Ask: {ask_price:.5f} | "
                                      f"Mid: {mid_price:.5f} | Spread: {spread_pips:.1f} pips")
                            
                            # Call callback function if provided
                            if callback:
                                try:
                                    callback(timestamp, instrument_name, bid_price, ask_price, mid_price)
                                except Exception as e:
                                    print(f"⚠️  Callback error: {e}")
                            
                            # Yield price data for generator usage
                            yield price_data
                            
                    elif data.get('type') == 'HEARTBEAT':
                        # Heartbeat to keep connection alive
                        if price_count % 100 == 0:  # Print occasionally
                            print(f"💓 Heartbeat - Connection alive ({price_count} prices received)")
                    
                    else:
                        # Other message types
                        print(f"📨 Message: {data.get('type', 'Unknown')} - {data}")
                        
                except json.JSONDecodeError as e:
                    print(f"⚠️  JSON decode error: {e}")
                    continue
                except Exception as e:
                    print(f"⚠️  Processing error: {e}")
                    continue
        
        print(f"\n✅ Streaming completed. Total prices received: {price_count}")
        
    except KeyboardInterrupt:
        print(f"\n🛑 Streaming stopped by user. Total prices received: {price_count}")
    except requests.exceptions.Timeout:
        print("❌ TIMEOUT ERROR: Streaming request timed out")
    except requests.exceptions.ConnectionError:
        print("❌ CONNECTION ERROR: Lost connection to OANDA")
    except Exception as e:
        print(f"❌ UNEXPECTED ERROR: {e}")
    finally:
        if 'response' in locals():
            response.close()

print("✅ OANDA live streaming functions ready!")
print("\n🔧 USAGE EXAMPLES:")
print("# Stream indefinitely:")
print("for price in stream_oanda_live_prices(api_key, account_id, 'EUR_USD'):")
print("    print(f\"EUR/USD: {price['price']:.5f}\")")
print("\n# Collect prices for 60 seconds:")
print("prices = collect_streaming_prices(api_key, account_id, 'GBP_JPY', 60)")


# Function: Get precision from a decimal number

In [ ]:
def get_precision(num):
    """Get precision by converting to string and counting decimal places"""
    num_str = str(num)
    if '.' in num_str:
        return len(num_str.split('.')[1])
    return 0

# Function: Plot with Matplotlib

In [ ]:
import matplotlib.pyplot as plt

def plot_time_series(df, exclude_columns=None):
    """
    Plot all columns except timestamp and excluded columns dynamically
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with timestamp column and data columns to plot
    exclude_columns : list or None
        List of column names to exclude from plotting (in addition to 'timestamp')
    """
    if exclude_columns is None:
        exclude_columns = []
    
    # Always exclude timestamp, plus any user-specified columns
    all_excluded = ['timestamp'] + exclude_columns
    
    plt.figure(figsize=(15, 8))
    
    # Find all columns except excluded ones
    plot_columns = [col for col in df.columns if col not in all_excluded]
    colors = ['#dddddd', 'blue', 'purple', 'green', 'purple', 'orange', 'brown', 'pink', 'olive', 'cyan', 'magenta']
    
    for i, col in enumerate(plot_columns):
        color = colors[i % len(colors)]  # Cycle through colors
        # Make first column (usually price) more prominent
        linewidth = 1.5 if i == 0 else 2
        alpha = 0.8 if i == 0 else 1.0
        plt.plot(df['timestamp'], df[col], 
                 color=color, linewidth=linewidth, alpha=alpha, label=col)
    
    plt.title('Time Series Data - All Columns')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    if exclude_columns:
        print(f"Excluded columns: {exclude_columns}")
    print(f"Plotted {len(plot_columns)} columns: {plot_columns}")

# Function: Plot with Bokeh

In [ ]:
# Show the viz for df using bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Category10, Set3, Dark2
output_notebook()

def plot_bokeh_time_series(df, title='Time Series Data', x_axis_label='Time', y_axis_label='Value'):
    """
    Plot time series data using Bokeh for interactive visualization.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame with 'timestamp' and data columns to plot
    title : str
        Title of the plot
    x_axis_label : str
        Label for the x-axis
    y_axis_label : str
        Label for the y-axis
    """
    source = ColumnDataSource(df)
    
    p = figure(title=title, x_axis_type='datetime', width=800, height=400,
               x_axis_label=x_axis_label, y_axis_label=y_axis_label)
    
    # Define color palette - combines multiple palettes for more variety
    colors = [
        '#b9babc', 
        'blue',
        'purple',
        '#ff7f0e',  # orange  
        '#2ca02c',  # green
        '#d62728',  # red
        '#9467bd',  # purple
        '#8c564b',  # brown
        '#e377c2',  # pink
        '#7f7f7f',  # gray
        '#bcbd22',  # olive
        '#17becf',  # cyan
        '#aec7e8',  # light blue
        '#ffbb78',  # light orange
        '#98df8a',  # light green
        '#ff9896',  # light red
        '#c5b0d5'   # light purple
    ]
    
    # Get columns to plot (exclude timestamp)
    plot_columns = [col for col in df.columns if col != 'timestamp']
    
    # Plot each column with its own color
    for i, col in enumerate(plot_columns):
        color = colors[i % len(colors)]  # Cycle through colors if more columns than colors
        
        # Different line styles for better distinction
        line_width = 3 if col == 'price' else 2  # Make price line thicker if it exists
        line_alpha = 0.9 if col == 'price' else 0.8
        
        p.line('timestamp', col, source=source, 
               legend_label=col, 
               line_width=line_width, 
               color=color,
               alpha=line_alpha)
    
    # Add hover tool with dynamic tooltips for all columns
    hover_tooltips = [('Time', '@timestamp{%F %T}')]
    for col in plot_columns:
        hover_tooltips.append((col, f'@{col}{{0.00000}}'))
    
    hover = HoverTool()
    hover.tooltips = hover_tooltips
    hover.formatters = {'@timestamp': 'datetime'}
    p.add_tools(hover)
    
    p.legend.location = 'top_left'
    p.legend.click_policy = "hide"  # Allow hiding lines by clicking legend
    p.grid.grid_line_alpha = 0.3
    
    show(p)


# Class: Time-based Streaming Moving Averages (EMA DEMA TEMA)

In [ ]:
# 🕐 TIME-BASED STREAMING MOVING AVERAGES CLASS
from collections import deque
import pandas as pd
from datetime import datetime, timedelta

class TimeBasedStreamingMA:
    """
    Time-based streaming moving average calculator that uses actual timestamps
    instead of fixed number of data points. Supports SMA, EMA, DEMA, and TEMA.
    
    Key Features:
    - Uses time windows (e.g., "5 minutes", "1 hour") instead of row counts
    - Automatically handles irregular time intervals
    - Maintains time-weighted calculations
    - Supports all four MA types with time-based logic
    """
    
    def __init__(self, time_window, ma_type='SMA', alpha=None):
        """
        Initialize the time-based streaming moving average calculator.
        
        Parameters:
        -----------
        time_window : str or timedelta
            Time window for calculations (e.g., '5min', '1H', '30s')
            Can be pandas timedelta string or datetime.timedelta object
        ma_type : str
            Type of moving average: 'SMA', 'EMA', 'DEMA', 'TEMA'
        alpha : float, optional
            Smoothing factor for EMA. If None, calculated based on time window
        """
        self.ma_type = ma_type.upper()
        
        # Validate MA type
        if self.ma_type not in ['SMA', 'EMA', 'DEMA', 'TEMA']:
            raise ValueError("ma_type must be one of: 'SMA', 'EMA', 'DEMA', 'TEMA'")
        
        # Convert time window to timedelta
        if isinstance(time_window, str):
            # Handle common abbreviations and deprecated formats
            time_window_str = time_window.replace('H', 'h')  # Fix deprecated 'H' format
            self.time_window = pd.Timedelta(time_window_str)
        elif isinstance(time_window, timedelta):
            self.time_window = pd.Timedelta(time_window)
        else:
            raise ValueError("time_window must be a string (e.g., '5min') or timedelta object")
        
        # Store original time window specification
        self.time_window_str = str(time_window)
        
        # Calculate alpha for EMA-based calculations
        if alpha is None:
            # Convert time window to approximate number of periods for alpha calculation
            # Assume 1-minute base period for alpha calculation
            minutes = self.time_window.total_seconds() / 60
            equivalent_periods = max(1, minutes)  # At least 1 period
            self.alpha = 2.0 / (equivalent_periods + 1)
        else:
            if not (0 < alpha < 1):
                raise ValueError("alpha must be between 0 and 1")
            self.alpha = alpha
        
        # Initialize data storage - we need to keep all data within time window for SMA
        self.data_points = deque()  # Store (timestamp, price) tuples
        self.timestamps = deque()   # Store just timestamps for quick access
        
        # For EMA, DEMA, TEMA - maintain running calculations
        if self.ma_type != 'SMA':
            self.ema1 = None  # First EMA
            self.ema2 = None  # Second EMA (for DEMA, TEMA)
            self.ema3 = None  # Third EMA (for TEMA)
            self.initialized = False
            self.last_timestamp = None
        
        self.data_count = 0
        
    def _clean_old_data(self, current_timestamp):
        """Remove data points older than the time window."""
        cutoff_time = current_timestamp - self.time_window
        
        # Remove old data points
        while self.data_points and self.data_points[0][0] < cutoff_time:
            self.data_points.popleft()
            if self.timestamps:
                self.timestamps.popleft()
    
    def _calculate_time_weight(self, current_timestamp, last_timestamp):
        """Calculate time-based weight for EMA calculations."""
        if last_timestamp is None:
            return self.alpha
        
        # Calculate time elapsed in seconds
        time_elapsed = (current_timestamp - last_timestamp).total_seconds()
        
        # Handle edge cases
        if time_elapsed <= 0:
            return self.alpha  # No time elapsed, use base alpha
        
        # Assume base interval for alpha calculation (e.g., 60 seconds)
        base_interval = 60.0  # 1 minute
        
        # Adjust alpha based on actual time elapsed
        time_factor = time_elapsed / base_interval
        
        # Prevent issues with very small alpha values and large time factors
        if self.alpha <= 0 or self.alpha >= 1:
            return self.alpha
        
        # Apply time-weighted alpha: more time elapsed = more weight to new data
        try:
            adjusted_alpha = 1 - (1 - self.alpha) ** time_factor
            return min(1.0, max(0.0, adjusted_alpha))  # Clamp between 0 and 1
        except (ZeroDivisionError, OverflowError, ValueError):
            # Fallback to base alpha if calculation fails
            return self.alpha
    
    def add_data_point(self, timestamp, price):
        """
        Add a new data point and calculate the updated time-based moving average.
        
        Parameters:
        -----------
        timestamp : datetime or str
            Timestamp of the data point
        price : float
            Price value
            
        Returns:
        --------
        dict
            Dictionary containing current MA, time window info, and metadata
        """
        # Convert timestamp to datetime if it's a string
        if isinstance(timestamp, str):
            timestamp = pd.to_datetime(timestamp)
        
        self.data_count += 1
        
        # Clean old data points outside time window
        self._clean_old_data(timestamp)
        
        # Add new data point
        self.data_points.append((timestamp, price))
        self.timestamps.append(timestamp)
        
        if self.ma_type == 'SMA':
            return self._calculate_time_sma(timestamp, price)
        elif self.ma_type == 'EMA':
            return self._calculate_time_ema(timestamp, price)
        elif self.ma_type == 'DEMA':
            return self._calculate_time_dema(timestamp, price)
        elif self.ma_type == 'TEMA':
            return self._calculate_time_tema(timestamp, price)
    
    def _calculate_time_sma(self, timestamp, price, return_full_window=False):
        """Calculate time-based Simple Moving Average."""
        # Calculate SMA using all data points within time window
        if len(self.data_points) == 0:
            current_ma = price
        else:
            total_price = sum(p for t, p in self.data_points)
            current_ma = total_price / len(self.data_points)
        
        if return_full_window:
            return {
                'timestamp': timestamp,
                'price': price,
                'moving_average': current_ma,
                'data_points_count': len(self.data_points),
                'time_window': self.time_window_str,
                'window_start': self.timestamps[0] if self.timestamps else timestamp,
                'window_end': timestamp,
                'time_span_actual': (timestamp - self.timestamps[0]).total_seconds() if self.timestamps else 0,
                'time_span_target': self.time_window.total_seconds(),
                'is_full_window': (timestamp - self.timestamps[0]) >= self.time_window if self.timestamps else False,
                'ma_type': 'Time-SMA'
            }
        return current_ma
    
    def _calculate_time_ema(self, timestamp, price, return_full_window=False):
        """Calculate time-based Exponential Moving Average."""
        if not self.initialized:
            # Initialize with first price
            self.ema1 = price
            self.initialized = True
            self.last_timestamp = timestamp
            time_weight = self.alpha
        else:
            # Calculate time-adjusted alpha
            time_weight = self._calculate_time_weight(timestamp, self.last_timestamp)
            # EMA calculation with time weighting
            self.ema1 = time_weight * price + (1 - time_weight) * self.ema1
            self.last_timestamp = timestamp
        
        if return_full_window:
            return {
                'timestamp': timestamp,
                'price': price,
                'moving_average': self.ema1,
                'data_points_count': self.data_count,
                'time_window': self.time_window_str,
                'window_start': self.timestamps[0] if self.timestamps else timestamp,
                'window_end': timestamp,
                'time_span_actual': (timestamp - self.timestamps[0]).total_seconds() if self.timestamps else 0,
                'time_span_target': self.time_window.total_seconds(),
                'is_full_window': (timestamp - self.timestamps[0]) >= self.time_window if self.timestamps else False,
                'ma_type': 'Time-EMA',
                'alpha_used': time_weight,
                'base_alpha': self.alpha
        }
        return self.ema1
        
    def _calculate_time_dema(self, timestamp, price, return_full_window=False):
        """Calculate time-based Double Exponential Moving Average."""
        if not self.initialized:
            # Initialize with first price
            self.ema1 = price
            self.ema2 = price
            self.initialized = True
            self.last_timestamp = timestamp
            time_weight = self.alpha
        else:
            # Calculate time-adjusted alpha
            time_weight = self._calculate_time_weight(timestamp, self.last_timestamp)
            # First EMA
            self.ema1 = time_weight * price + (1 - time_weight) * self.ema1
            # Second EMA (EMA of first EMA)
            self.ema2 = time_weight * self.ema1 + (1 - time_weight) * self.ema2
            self.last_timestamp = timestamp
        
        # DEMA = 2 * EMA1 - EMA2
        dema = 2 * self.ema1 - self.ema2
        
        if return_full_window:
            return {
                'timestamp': timestamp,
                'price': price,
                'moving_average': dema,
                'data_points_count': self.data_count,
                'time_window': self.time_window_str,
                'window_start': self.timestamps[0] if self.timestamps else timestamp,
                'window_end': timestamp,
                'time_span_actual': (timestamp - self.timestamps[0]).total_seconds() if self.timestamps else 0,
                'time_span_target': self.time_window.total_seconds(),
                'is_full_window': (timestamp - self.timestamps[0]) >= self.time_window if self.timestamps else False,
                'ma_type': 'Time-DEMA',
                'alpha_used': time_weight,
                'base_alpha': self.alpha,
                'ema1': self.ema1,
                'ema2': self.ema2
            }
        return dema
    
    def _calculate_time_tema(self, timestamp, price, return_full_window=False):
        """Calculate time-based Triple Exponential Moving Average."""
        if not self.initialized:
            # Initialize with first price
            self.ema1 = price
            self.ema2 = price
            self.ema3 = price
            self.initialized = True
            self.last_timestamp = timestamp
            time_weight = self.alpha
        else:
            # Calculate time-adjusted alpha
            time_weight = self._calculate_time_weight(timestamp, self.last_timestamp)
            # First EMA
            self.ema1 = time_weight * price + (1 - time_weight) * self.ema1
            # Second EMA (EMA of first EMA)
            self.ema2 = time_weight * self.ema1 + (1 - time_weight) * self.ema2
            # Third EMA (EMA of second EMA)
            self.ema3 = time_weight * self.ema2 + (1 - time_weight) * self.ema3
            self.last_timestamp = timestamp
        
        # TEMA = 3 * EMA1 - 3 * EMA2 + EMA3
        tema = 3 * self.ema1 - 3 * self.ema2 + self.ema3
        
        if return_full_window:
            return {
                'timestamp': timestamp,
                'price': price,
                'moving_average': tema,
                'data_points_count': self.data_count,
                'time_window': self.time_window_str,
                'window_start': self.timestamps[0] if self.timestamps else timestamp,
                'window_end': timestamp,
                'time_span_actual': (timestamp - self.timestamps[0]).total_seconds() if self.timestamps else 0,
                'time_span_target': self.time_window.total_seconds(),
                'is_full_window': (timestamp - self.timestamps[0]) >= self.time_window if self.timestamps else False,
                'ma_type': 'Time-TEMA',
                'alpha_used': time_weight,
                'base_alpha': self.alpha,
                'ema1': self.ema1,
                'ema2': self.ema2,
                'ema3': self.ema3
            }
        return tema
    
    def get_current_ma(self):
        """Get the current moving average without adding new data."""
        if self.ma_type == 'SMA':
            if len(self.data_points) == 0:
                return None
            total_price = sum(p for t, p in self.data_points)
            return total_price / len(self.data_points)
        else:
            if not self.initialized:
                return None
            if self.ma_type == 'EMA':
                return self.ema1
            elif self.ma_type == 'DEMA':
                return 2 * self.ema1 - self.ema2
            elif self.ma_type == 'TEMA':
                return 3 * self.ema1 - 3 * self.ema2 + self.ema3
    
    def get_time_window_info(self):
        """Get information about the current time window state."""
        current_time = self.timestamps[-1] if self.timestamps else None
        oldest_time = self.timestamps[0] if self.timestamps else None
        
        base_info = {
            'ma_type': f'Time-{self.ma_type}',
            'time_window_spec': self.time_window_str,
            'time_window_seconds': self.time_window.total_seconds(),
            'data_points_count': len(self.data_points),
            'total_data_processed': self.data_count,
            'current_ma': self.get_current_ma(),
            'oldest_timestamp': oldest_time,
            'newest_timestamp': current_time,
            'actual_time_span': (current_time - oldest_time).total_seconds() if current_time and oldest_time else 0,
            'window_utilization': ((current_time - oldest_time).total_seconds() / self.time_window.total_seconds() * 100) if current_time and oldest_time else 0
        }
        
        if self.ma_type != 'SMA':
            base_info.update({
                'base_alpha': self.alpha,
                'initialized': self.initialized,
                'last_calculation_time': self.last_timestamp
            })
            
            if self.initialized:
                if self.ma_type in ['EMA', 'DEMA', 'TEMA']:
                    base_info['ema1'] = self.ema1
                if self.ma_type in ['DEMA', 'TEMA']:
                    base_info['ema2'] = self.ema2
                if self.ma_type == 'TEMA':
                    base_info['ema3'] = self.ema3
        
        return base_info
    
    def reset(self):
        """Reset the moving average calculator."""
        self.data_count = 0
        self.data_points.clear()
        self.timestamps.clear()
        
        if self.ma_type != 'SMA':
            self.ema1 = None
            self.ema2 = None
            self.ema3 = None
            self.initialized = False
            self.last_timestamp = None

# Convenience functions for creating time-based streaming MAs
def create_time_sma(time_window):
    """Create a time-based Simple Moving Average calculator."""
    return TimeBasedStreamingMA(time_window, ma_type='SMA')

def create_time_ema(time_window, alpha=None):
    """Create a time-based Exponential Moving Average calculator."""
    return TimeBasedStreamingMA(time_window, ma_type='EMA', alpha=alpha)

def create_time_dema(time_window, alpha=None):
    """Create a time-based Double Exponential Moving Average calculator."""
    return TimeBasedStreamingMA(time_window, ma_type='DEMA', alpha=alpha)

def create_time_tema(time_window, alpha=None):
    """Create a time-based Triple Exponential Moving Average calculator."""
    return TimeBasedStreamingMA(time_window, ma_type='TEMA', alpha=alpha)

print("✅ TimeBasedStreamingMA class defined successfully!")

# <font color=pink>Placeholder</font>

In [ ]:
# Placeholder

# Execution: Get Data from 1 of 3 sources: Monte Carlo, OANDA and Database

In [135]:
import numpy as np
import json

# Load secrets from secrets.json
with open('secrets.json', 'r') as f:
    secrets = json.load(f)

api_key = secrets['api_key']
account_id = secrets['account_id']

# Generate the data with 5 seconds intervals for 23 hours
interval = 5  # 1 second intervals
hours = 23  
np.random.seed(42)  # For reproducible results

SOURCE = 2 # 1 = Monte Carlo, 2 = OANDA, 3 = Database

# Generate stock price data using Monte Carlo method    
if SOURCE == 1:
    print("Generating stock price data using Monte Carlo method...")
    hours = int(input("Number of hours to fetch (e.g., 10): "))
    stock_df = generate_stock_price_data(interval=5, hours=hours) # 1-second intervals

if SOURCE == 2:
    # Generate stock data from OANDA API
    instrument = input("Instrument (e.g., USD_CAD): ")
    granularity = 'S5' # input("Granularity (e.g., S5, M1, M5): ")
    hours = int(input("Number of hours to fetch (e.g., 10): "))
    stock_df = get_oanda_data(
        api_key=api_key,
        account_id=account_id,
        instrument=instrument,
        granularity=granularity,
        hours=hours
    )

if SOURCE == 3:
    conn = connect_to_database()
    instrument = input("Enter instrument name to fetch data (e.g., 'USD_CAD'): ") or 'USD_CAD'
    rows = int(input("Enter number of rows to fetch (e.g., 10000): ") or 10000)
    offset = int(input("Enter offset in hours (e.g., 0): ") or 0)

    # query timestamp with n hours of offset on the date
    query = f"SELECT datetime(date, '-{offset} hours') AS timestamp, buy AS price FROM gather WHERE instrument = '{instrument}' ORDER BY timestamp LIMIT {rows}"
    stock_df = pd.read_sql_query(query, conn)
    close_database_connection(conn)

    stock_df['timestamp'] = pd.to_datetime(stock_df['timestamp'])
    stock_df = stock_df.drop_duplicates(subset=['timestamp'], keep='last')

    # Remove rows where for a given day there is less than 12 rows
    stock_df = stock_df.groupby(stock_df['timestamp'].dt.date).filter(lambda x: len(x) >= 12)

    # Resample to 5-second intervals, forward filling missing values
    stock_df.set_index('timestamp', inplace=True)
    stock_df = stock_df.resample('5s').ffill().reset_index()



# from stock_df, create different intervals
stock_5sec = stock_df.copy()
stock_1min = stock_df[::12].reset_index(drop=True)  # Every 12th row for 1-minute intervals
stock_5min =  stock_df[::60].reset_index(drop=True)  # Every 60th row for 5-minute intervals
stock_10min = stock_df[::120].reset_index(drop=True)  # Every 120th row for 10-minute intervals
stock_15min = stock_df[::180].reset_index(drop=True)  # Every 180th row for 15-minute intervals
stock_1hour = stock_df[::720].reset_index(drop=True)  # Every 720th row for 1-hour intervals
stock_variable = create_variable_interval_data(stock_df, min_interval=1, max_interval=2) # variable intervals


for df, label in zip([stock_variable, stock_5sec, stock_1min, stock_5min, stock_10min, stock_15min, stock_1hour], 
                   ['Variable', '5-second', '1-minute', '5-minute', '10-minute', '15-minute', '1-hour']):
    plot_bokeh_time_series(df, title='Stock Price Data')


KeyboardInterrupt: Interrupted by user

# Execution: Choose your DF

In [136]:
df = None
df = stock_5sec.copy()  # Use the 5-second interval data for EMA calculation
# df = stock_1min.copy()  # Use the 1-minute interval data for EMA calculation
# df = stock_5min.copy()  # Use the 5-minute interval data for EMA calculation
# df = stock_1hour.copy()  # Use the 1-hour interval
# df = stock_variable.copy()  # Use the variable interval

# remove nan values
df = df.dropna(subset=['price'])

# save the data to a CSV file
df.to_csv('df.csv', index=False)

plot_bokeh_time_series(df, title='Stock Price Data')


# Execution: Add EMAs to entire Data Frame

In [ ]:

# Add EMA to the DataFrame
df['EMA_15min'] = calculate_time_based_ema(df, type=1, period_minutes=15)
# df['DEMA_15min'] = calculate_time_based_ema(df, type=2, period_minutes=15)
df['TEMA_15min'] = calculate_time_based_ema(df, type=3, period_minutes=15)

plot_bokeh_time_series(df, title='Stock Price Data')

# stock_df

# Execution: Process EMAs using streaming

In [ ]:
import time

# Clean df
df = df[['timestamp', 'price']].copy()  # Keep only necessary columns

# 1. Create the calculator
ema_calc = TimeBasedStreamingMA('15min', ma_type='EMA')
tema_calc = TimeBasedStreamingMA('15min', ma_type='TEMA')

# initialize the json that will hold timestamp price and ema values
results_json = [] # list of dictionaries to hold results
ema_values = []
tema_values = []

# 2. Process the DataFrame
for _, row in df.iterrows():
    # wait 1 second to simulate real-time processing
    # time.sleep(1)  # Uncomment this line to simulate real-time processing
    timestamp = row['timestamp']
    price = row['price']

    # only from the first price, infer the precision
    if not ema_values:  # Only do this once
        precision = get_precision(price)

    ema = round(ema_calc.add_data_point(timestamp, price), precision)
    tema = round(tema_calc.add_data_point(timestamp, price), precision)
    ema_values.append(ema)  # EMA value
    tema_values.append(tema)  # TEMA value

    # Check for crossovers and store results in JSON format
    cross_direction = None
    
    # when tema crosses ema, detect the direction
    if len(ema_values) > 1 and len(tema_values) > 1:
        if (tema_values[-1] > ema_values[-1] and tema_values[-2] <= ema_values[-2]):
            cross_direction = 'up'
            print('up', timestamp, tema_values[-1], ema_values[-1])  # Print timestamp and values
        elif (tema_values[-1] < ema_values[-1] and tema_values[-2] >= ema_values[-2]):
            cross_direction = 'down'
            print('down', timestamp, tema_values[-1], ema_values[-1])  # Print timestamp and values

    # Store results in JSON format - include cross info only when crossing occurs
    result_entry = {
        "timestamp": timestamp.isoformat(),
        "price": price,
        "ema": ema,
        "tema": tema
    }
    
    # Add cross information only when there's a crossing
    if cross_direction:
        result_entry["cross"] = cross_direction
        result_entry["cross_timestamp"] = timestamp.isoformat()

    # add result_entry to a live plot and display it
    results_json.append(result_entry)
    
# 3. Add to DataFrame
df['EMA_15min'] = ema_values
df['TEMA_15min'] = tema_values
# df['Other_15min'] = other_values  # If you have other values to track

# Inspect object
print(f'{len(df) = }') 
print(f'{len(ema_calc.timestamps) = }')
print(f'{len(ema_calc.data_points) = }')
print(f'{ema_calc.data_count = }')
print(f'{ema_calc.ema1 = }')
print(f'{ema_calc.ema2 = }')
print(f'{ema_calc.ema3 = }')
print(f'{ema_calc.last_timestamp = }')
print(f'{ema_calc.ma_type = }')
print(f'{ema_calc.time_window = }')
print(f'{ema_calc.time_window_str = }')
print(f'{ema_calc.alpha = }')
print(f'{ema_calc.initialized = }')
ema_calc.get_time_window_info()

plot_bokeh_time_series(df, title='Stock Price Data')

# Execution: OANDA Live Streaming Prices

In [ ]:
import json

# Load secrets from secrets.json
with open('secrets.json', 'r') as f:
    secrets = json.load(f)

api_key = secrets['api_key']
account_id = secrets['account_id']
instrument = input("Instrument (e.g., USD_CAD): ")

for price in stream_oanda_live_prices(api_key, account_id, instrument):
    # print(f"{instrument}: {price['price']:.10f}")
    print(price['bid'])



# <font color=lightgreen>Execution:</font> OANDA Get Instrument Precision

In [ ]:
with open('secrets.json', 'r') as f:
    secrets = json.load(f)

url = secrets['url']
api_key = secrets['api_key']
account_id = secrets['account_id']

instrument = input("Enter the instrument name (e.g., USD_CAD): ") or 'USD_CAD'
precision = get_instrument_precision(url, api_key, account_id, instrument)
precision

# <font color=lightgreen>Execution:</font> OANDA Create Stop Order

In [ ]:
# Example: Create a stop order to buy 100 units of EUR_USD at 1.1000
# In a real scenario, you would replace the placeholder credentials in the function.
# Load secrets from secrets.json
with open('secrets.json', 'r') as f:
    secrets = json.load(f)


url = secrets['url']
api_key = secrets['api_key']
account_id = secrets['account_id']

instrument = input("Enter the instrument name (e.g., USD_CAD): ")
action = input("Enter the action (buy/sell): ").lower()
amount = float(input("Enter the amount (e.g., 160): "))
price = float(input("Enter the price (e.g., 1.36400): "))

assert instrument and action and amount and price, "All inputs must be provided."

if action == 'buy':
    order = create_stop_order(url, api_key, account_id, instrument, amount, price)
elif action == 'sell':
    order = create_stop_order(url, api_key, account_id, instrument, -amount, price)
order


# <font color=lightgreen>Execution:</font> OANDA Cancel Orders 

In [ ]:
with open('secrets.json', 'r') as f:
    secrets = json.load(f)

url = secrets['url']
api_key = secrets['api_key']
account_id = secrets['account_id']

cancel_orders(url, api_key, account_id, 0)  # Cancel all stop orders for the specified instrument

# Execution: OANDA Get Current Position

In [ ]:
import json

with open('secrets.json', 'r') as f:
    secrets = json.load(f)

url = secrets['url']
api_key = secrets['api_key']
account_id = secrets['account_id']
instrument = input("Enter the instrument name (e.g., USD_CAD): ") or 'USD_CAD'

position = get_instrument_position(url, api_key, account_id, instrument)
position

# Execution: OANDA Get list of orders

In [ ]:
import json

with open('secrets.json', 'r') as f:
    secrets = json.load(f)

url = secrets['url']
api_key = secrets['api_key']
account_id = secrets['account_id']
instrument = input("Enter the instrument name (e.g., USD_CAD): ") or 'USD_CAD'

list_of_orders = list_orders(url, api_key, account_id)
list_of_orders


# Execution: OANDA Get list of trades

In [ ]:
import json

with open('secrets.json', 'r') as f:
    secrets = json.load(f)

url = secrets['url']
api_key = secrets['api_key']
account_id = secrets['account_id']
instrument = input("Enter the instrument name (e.g., USD_CAD): ") or 'USD_CAD'

list_of_trades = list_trades(url, api_key, account_id)
list_of_trades

# Placeholder